In [55]:
import pandas as pd
import numpy as np
import scipy.optimize as sc
import matplotlib.pyplot as plt
import json

# Обработка данных из таблиц ресурсов и использования товаров и услуг

In [65]:
def read_and_formalize_tri_data(file_name: str = '../data/raw_data/TRI_2019.xlsx',
                                sheet_name: str = 'М-отеч',
                                header: int = 3):
    """
    Reads data from file of '../data/raw_data/TRI_2019.xlsx' structure.
    Aggregates inputs and outputs.
    
    Input:
    ------
    str: file_name - name of the file with TRI data
    str: sheet_name - name of the shhet needed
    int: header - amount of header-lines without necessary data
    
    Output:
    -------
    pd.DataFrame: df - formalized data from the input file with
                       aggregated inputs and outputs
    """
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=header)
    df = df.drop(['Unnamed: 1', 'Unnamed: 2'], axis=1).set_index('№№')
    df.columns = df.columns.astype(int)
    
    idx = list(np.arange(1, 66)) + [72]
    col = list(np.arange(1, 62)) + [71]
    
    df = (
        df
        .loc[idx, col]
        .append(pd.DataFrame(df.loc[62:65, col].sum(),
                             columns=['rest_production']).transpose())
        .rename(columns={71: 'utilisation'},
                index={72: 'VDS'})
        .drop([62, 63, 64, 65], axis=0)
    )
    return df

def agregate_array_by_14_branches(array_inp: np.array,
                                  idx_by_line: list):
    """
    Aggregates data in the input array.
    
    Input:
    ------
    np.array: array_inp - data for agregation,
    list: idx_by_line - list of indeces for aggregation in each line
    
    Output:
    -------
    np.array: array_agg - aggregated data
    """
    n_lines = len(idx_by_line)
    array_agg = np.zeros(n_lines)
    for i in range(0, n_lines):
        idx_line = idx_by_line[i]
        for idx in idx_line:
            if idx['begin'] != idx['end']:
                array_agg[i] += array_inp[idx['begin'] : idx['end']].sum()
            else:
                array_agg[i] += array_inp[idx['begin']]
    return np.array([array_agg]) 

def set_transition_matrix(idx_by_line: list,
                          n_cols: int = 61):
    """
    Sets transition matrix for data aggregation.
    
    Input:
    ------
    list: idx_by_line - list of indeces for aggregation in each line,
    int: n_cols - number of columns for the output matrix
    
    Output:
    -------
    np.array: T - transition matix
    """
    n_lines = len(idx_by_line)
    T = np.zeros((n_lines, n_cols))
    for i in range(0, n_lines):
        idx_line = idx_by_line[i]
        for idx in idx_line:
            if idx['begin'] != idx['end']:
                T[i, idx['begin']:idx['end']] = np.ones(idx['end'] - idx['begin'])
            else:
                T[i, idx['begin']] = 1
    return T

def preprocess_tri_data(file_raw: str = '../data/raw_data/TRI_2019.xlsx',
                        file_algo: str = '../data/algo_data/transition_matrix_idx_by_line.json'):
    """
    Processes data from the input file of the given format.
    The format from '../data/raw_data/TRI_2019.xlsx' file,
    updates can be downloaded via  https://rosstat.gov.ru/statistics/accounts
    
    Input:
    ------
    str: file_raw - path to file with raw TRI data
    str: file_algo - path_to_file with idx_by_line data
    
    Output:
    -------
    pd.DataFrame: df_short - preprocessed dataframe for further analysis
    """
    df = read_and_formalize_tri_data(file_raw)
    
    f = open(file_algo)
    idx_by_line = json.load(f)
    f.close()

    T = set_transition_matrix(idx_by_line)

    vds = np.array(df.loc['VDS'])[:-1]
    vds_short = agregate_array_by_14_branches(vds, idx_by_line)

    rest_production = np.array(df.loc['rest_production'])[:-1]
    rest_production_short = agregate_array_by_14_branches(rest_production, idx_by_line)

    short_mat = T_short @ np.array(df)[:61, :61] @ T_short.T

    utilisation_short = short_mat.sum(0) + vds_short + \
                        rest_production_short - short_mat.sum(1)
    utilisation_short = np.array([np.append(utilisation_short, [0, 0])])

    short_mat = np.concatenate([short_mat, vds_short, rest_production_short], axis=0)
    short_mat = np.concatenate([short_mat.T, utilisation_short]).T

    df_short = pd.DataFrame(short_mat)
    df_short.columns = list(np.arange(1, 15)) + ['utilisation']
    df_short.index = list(np.arange(1, 15)) + ['vds', 'rest_production']
    
    return df_short

In [235]:
df_2020 = preprocess_tri_data('../data/raw_data/TRI_2020.xlsx')
df_2019 = preprocess_tri_data('../data/raw_data/TRI_2019.xlsx')

/tmp/ipykernel_334337/3864757320.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df
/tmp/ipykernel_334337/3864757320.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df


In [236]:
df_2020

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,utilisation
1,1352417.0,71.0,2688536.0,3809.0,104.0,13464.0,12460.0,7624.0,46193.0,54.0,513.0,7836.0,10665.0,40024.0,4067441.0
2,4990.0,1044593.0,4149255.0,904464.0,2685.0,175369.0,257711.0,142248.0,176.0,55.0,4388.0,970.0,1047.0,15448.0,7666181.0
3,1199448.0,826481.0,12867351.0,407843.0,316743.0,3277808.0,725115.0,1643138.0,350261.0,246045.0,367829.0,118575.0,508206.0,1091413.0,27322351.0
4,137469.0,363104.0,1562848.0,3312794.0,83060.0,67702.0,223864.0,385235.0,32806.0,59266.0,388871.0,122211.0,88605.0,317496.0,1593445.0
5,8746.0,19755.0,502829.0,60032.0,266895.0,20076.0,27870.0,18069.0,6786.0,3916.0,121065.0,15871.0,19760.0,106543.0,413141.0
6,23417.0,240836.0,226195.0,119627.0,36605.0,311451.0,61721.0,154495.0,10518.0,16508.0,322758.0,95627.0,118021.0,736459.0,10152924.0
7,325564.0,192542.0,3238842.0,619964.0,109183.0,763343.0,710674.0,482609.0,86184.0,62410.0,169503.0,42117.0,288569.0,370467.0,14346796.0
8,152487.0,1012054.0,2739656.0,96283.0,73480.0,262804.0,3135409.0,2372361.0,15630.0,43983.0,22174.0,14888.0,61579.0,744802.0,3684706.0
9,957.0,7910.0,24275.0,3743.0,466.0,27103.0,13743.0,16694.0,6867.0,8153.0,1592.0,66853.0,42491.0,111532.0,1328306.0
10,7742.0,30495.0,247332.0,59021.0,5053.0,41027.0,268013.0,115433.0,11228.0,1322316.0,57721.0,67259.0,30780.0,1026122.0,2503153.0


In [237]:
df_2019

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,utilisation
1,1287390.0,92.0,2816002.0,3724.0,60.0,15028.0,8461.0,7443.0,48795.0,57.0,429.0,7908.0,8421.0,46868.0,3299332.0
2,3398.0,1111753.0,5750397.0,882327.0,2799.0,174001.0,269490.0,126923.0,119.0,70.0,1227.0,558.0,1162.0,15112.0,9565751.0
3,1141376.0,883845.0,12487483.0,452105.0,262727.0,3317815.0,655179.0,1859379.0,347323.0,214216.0,331331.0,111813.0,360116.0,1118639.0,27811725.0
4,139192.0,389844.0,1514335.0,3194692.0,85950.0,78011.0,200326.0,416778.0,42645.0,56480.0,372180.0,162520.0,101382.0,318857.0,1643012.0
5,5864.0,14228.0,446466.0,60468.0,173868.0,16128.0,32457.0,16220.0,6559.0,4783.0,87396.0,18571.0,17454.0,91757.0,362541.0
6,22575.0,285736.0,228397.0,117016.0,26828.0,368356.0,59543.0,187963.0,16997.0,12334.0,395515.0,163337.0,112991.0,627559.0,9928523.0
7,292351.0,196230.0,3000262.0,744150.0,81356.0,774626.0,624948.0,448149.0,94369.0,64119.0,145224.0,36548.0,205444.0,373723.0,15277335.0
8,152337.0,1133614.0,2689110.0,77345.0,47154.0,332401.0,3058146.0,2530136.0,21620.0,50255.0,23045.0,19529.0,43008.0,697519.0,4339250.0
9,1156.0,8363.0,30813.0,5514.0,489.0,16976.0,22312.0,21206.0,4706.0,8771.0,1294.0,32813.0,41662.0,155583.0,1646754.0
10,7009.0,29166.0,218228.0,50409.0,4149.0,38361.0,248872.0,104385.0,9797.0,1105923.0,44833.0,55916.0,31853.0,902682.0,2345480.0


In [239]:
# df_short.to_latex()
df_2019.to_excel('../data/preprocessed_data/df_tri_2019.xlsx')
df_2020.to_excel('../data/preprocessed_data/df_tri_2020.xlsx')

/tmp/ipykernel_334337/1999815029.py:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df_2019.to_excel('../data/preprocessed_data/df_tri_2019.xlsx')
/tmp/ipykernel_334337/1999815029.py:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df_2020.to_excel('../data/preprocessed_data/df_tri_2020.xlsx')


# Обработка данных по занятости

In [109]:
def preprocess_labour_data(base_year: str = '2019',
                           file_raw: str = '../data/raw_data/non_tri_data.xlsx',
                           file_names: str = '../data/algo_data/branch_names_eng.json'):
    df_labour_by_region = pd.read_excel(file_raw)
    df_labour_by_region = (
        df_labour_by_region
        [['Unnamed: 0',
          'Численность рабочей силы, '+base_year+' (тыс. человек)']]
        .rename(columns = {
            'Unnamed: 0': 'Region',
            'Численность рабочей силы, '+base_year+' (тыс. человек)': 'Workforce'
        })
        .assign(Workforce_int = lambda df: df.Workforce.astype(int))
        .drop('Workforce', axis=1)
        .rename(columns={'Workforce_int': 'Workforce'})
    )
    
    f = open(file_names)
    industry_names = json.load(f)
    n_industry = len(industry_names)
    f.close()
    
    df_labour_by_industry = (
        pd.read_excel(file_raw, sheet_name='Labour_by_industry_'+base_year)
    )
    df_labour_by_industry.columns = ['Region'] + industry_names
    
    n_labour = np.array(df_labour_by_region['Workforce'])
    full_labour = np.array(df_labour_by_industry)[:, 1:] * \
                  np.tile(n_labour.T, (n_industry, 1)).T
    
    df_full_labour = pd.DataFrame(full_labour,
                              columns=industry_names,
                              index=df_labour_by_industry.index)
    df_full_labour.insert(0, 'Region', df_labour_by_industry['Region'])
    
    return df_full_labour, df_labour_by_region, df_labour_by_industry

In [240]:
df_full_labour_2019, _, _ = preprocess_labour_data('2019')
df_full_labour_2020, _, _ = preprocess_labour_data('2020')

In [117]:
df_full_labour

,Region,"Agriculture, forestry",Mining,Processing industries,Electric power supply and other,"Water supply, sanitation and other",Construction,Wholesale and retail trade,Transportation and storage,Operation of hotels and other,Information and communication,Operations with real estate,Education,Healthcare,Other activities
0,Российская Федерация,505166.6,120636.8,1055572.0,173415.4,75398.0,678582.0,1432562.0,573024.8,188495.0,158335.8,203574.6,573024.8,467467.6,1342084.4
1,Центральный федеральный округ,91744.8,8534.4,296570.4,38404.8,19202.4,209092.8,443788.8,160020.0,49072.8,57607.2,66141.6,132283.2,110947.2,450189.6
2,Белгородская область,10655.4,2891.0,13381.2,1404.2,1239.0,6442.8,14289.8,4708.2,1321.6,1156.4,1652.0,7681.8,5203.8,10738.0
3,Брянская область,5712.0,23.8,9579.5,1368.5,714.0,3629.5,11781.0,5117.0,1606.5,1130.5,1725.5,4343.5,4046.0,8746.5
4,Владимирская область,3677.1,216.3,17231.9,1802.5,793.1,6128.5,11608.1,4470.2,2018.8,1153.6,2379.3,4902.8,4253.9,11391.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Амурская область,2448.0,1591.2,2448.0,1754.4,448.8,6283.2,6568.8,4039.2,734.4,693.6,816.0,3100.8,2733.6,7058.4
87,Магаданская область,240.8,1229.8,266.6,550.4,60.2,593.4,1212.6,748.2,180.6,154.8,120.4,610.6,722.4,1909.2
88,Сахалинская область,1732.5,1045.0,1732.5,1017.5,302.5,3575.0,4290.0,2502.5,742.5,385.0,907.5,2145.0,2007.5,5087.5
89,Еврейская автономная область,616.0,238.7,623.7,300.3,84.7,492.8,1170.4,731.5,154.0,123.2,207.9,808.5,708.4,1447.6


In [241]:
df_full_labour_2019.to_excel('../data/preprocessed_data/df_labour_2019.xlsx', index=False)
df_full_labour_2020.to_excel('../data/preprocessed_data/df_labour_2020.xlsx', index=False)

/tmp/ipykernel_334337/3249281097.py:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df_full_labour_2019.to_excel('../data/preprocessed_data/df_labour_2019.xlsx', index=False)
/tmp/ipykernel_334337/3249281097.py:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df_full_labour_2020.to_excel('../data/preprocessed_data/df_labour_2020.xlsx', index=False)


# Обработка данных по ВДС

In [229]:
def preprocess_vds_data(base_year: str = '2019',
                        file_name: str = '../data/raw_data/non_tri_data.xlsx',
                        file_names: str = '../data/algo_data/branch_names_vds.json'):
    
    f = open(file_names)
    names_vds = json.load(f)
    f.close()
    df_vds = pd.read_excel(file_name,
                           sheet_name='VDS_by_industry_'+base_year).drop(['Всего'], axis=1)
    df_vds = (
        df_vds
        .rename(columns=dict(zip(df_vds.columns, names_vds)))
        .assign(Other = lambda df: df.eval('Financial + Science + Administration + Government + Culture + Households'))
        .drop(['Financial', 'Science', 'Administration', 'Government', 'Culture', 'Households', 'Other activities'], axis=1)
    )
    return df_vds

In [234]:
df_vds = preprocess_vds_data('2020')
# df_vds.to_excel('../data/preprocessed_data/df_vds_2020.xlsx')

# Обработка данных по ВВП

In [278]:
def preprocess_vvp_data(file_name: str = '../data/raw_data/vvp_2019_2020.xlsx',
                        file_names: str = '../data/algo_data/branch_names_vvp.json'):
    f = open('../data/algo_data/branch_names_vvp.json')
    names_vds = json.load(f)
    f.close()
    df_vvp = pd.read_excel('../data/raw_data/vvp_2019_2020.xlsx')
    df_vvp.insert(0, 'Branches', names_vds)
    df_vvp = (
        df_vvp
        .set_index('Branches')
        .drop('Unnamed: 0', axis=1)
        .transpose()
        .assign(Other = lambda df: df.eval('Financial + Science + Administration + Government + Culture + Households'))
        .drop(['Financial', 'Science', 'Administration', 'Government', 'Culture', 'Households', 'Other activities'], axis=1)
        .transpose()
        .assign(Proc = lambda df: df.eval('Y2020 / Y2019 * 100'))
    )
    return df_vvp

In [279]:
df_vvp = preprocess_vvp_data()
df_vvp

,Y2020,Y2019,Proc
Branches,,,
"Agriculture, forestry",3935.1,3791.2,103.795632
Mining,9393.8,12683.5,74.063153
Processing industries,14074.0,14127.5,99.621306
Electric power supply and other,2550.1,2562.9,99.500566
"Water supply, sanitation and other",484.4,478.2,101.296529
Construction,5458.1,5342.1,102.171431
Wholesale and retail trade,12499.5,12754.3,98.002242
Transportation and storage,6254.3,6708.1,93.235044
Operation of hotels and other,704.6,895.6,78.673515


In [281]:
# df_vvp.to_excel('../data/preprocessed_data/df_vvp.xlsx')